In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from gensim.models import KeyedVectors

In [ ]:
# Load the dataset
df = pd.read_csv('cleaned_infopankki-fa.csv')
english_sentences = df['English'].astype(str).tolist()
farsi_sentences = df['Persian'].astype(str).tolist()

In [ ]:
# English and Farsi tokenizers
tokenizer_en = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_fa = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
# Tokenize sentences
tokenized_en = [tokenizer_en.tokenize(sentence) for sentence in english_sentences]
tokenized_fa = [tokenizer_fa.tokenize(sentence) for sentence in farsi_sentences]

In [ ]:
# Build English and Farsi vocabularies
vocab_en = set()
for tokens in tokenized_en:
    vocab_en.update(tokens)

vocab_fa = set()
for tokens in tokenized_fa:
    vocab_fa.update(tokens)

In [ ]:
# Split the data into training and testing sets (80% training, 20% testing)
english_train_sentences, english_test_sentences, farsi_train_sentences, farsi_test_sentences = train_test_split(
    english_sentences, farsi_sentences, test_size=0.2, random_state=42)

In [ ]:
# Convert sentences into sequences of indices based on vocabulary
word2idx_en = {word: idx + 1 for idx, word in enumerate(vocab_en)}
word2idx_fa = {word: idx + 1 for idx, word in enumerate(vocab_fa)}
idx2word_fa = {idx: word for word, idx in word2idx_fa.items()}

In [ ]:
# Helper function to convert sentences into sequences of indices
def sentences_to_sequences(tokenizer, sentences, word2idx):
    tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    sequences = [[word2idx.get(token, 0) for token in tokens] for tokens in tokenized_sentences]
    return sequences

In [ ]:
# Convert training and test data to sequences
sequences_en_train = sentences_to_sequences(tokenizer_en, english_train_sentences, word2idx_en)
sequences_fa_train = sentences_to_sequences(tokenizer_fa, farsi_train_sentences, word2idx_fa)
sequences_en_test = sentences_to_sequences(tokenizer_en, english_test_sentences, word2idx_en)
sequences_fa_test = sentences_to_sequences(tokenizer_fa, farsi_test_sentences, word2idx_fa)

In [ ]:
# Get maximum sequence lengths
max_len_en = max(len(seq) for seq in sequences_en_train)
max_len_fa = max(len(seq) for seq in sequences_fa_train)

In [ ]:
# Pad the sequences
sequences_en_train_padded = pad_sequences(sequences_en_train, maxlen=max_len_en, padding='post')
sequences_fa_train_padded = pad_sequences(sequences_fa_train, maxlen=max_len_fa, padding='post')
sequences_en_test_padded = pad_sequences(sequences_en_test, maxlen=max_len_en, padding='post')
sequences_fa_test_padded = pad_sequences(sequences_fa_test, maxlen=max_len_fa, padding='post')

In [ ]:
# Prepare target sequences for the decoder (shifted Farsi sentences)
decoder_target_data_train = np.zeros_like(sequences_fa_train_padded)
decoder_target_data_train[:, :-1] = sequences_fa_train_padded[:, 1:]
decoder_target_data_train[:, -1] = 0
decoder_target_data_train = decoder_target_data_train.reshape(*decoder_target_data_train.shape, 1)

decoder_target_data_test = np.zeros_like(sequences_fa_test_padded)
decoder_target_data_test[:, :-1] = sequences_fa_test_padded[:, 1:]
decoder_target_data_test[:, -1] = 0
decoder_target_data_test = decoder_target_data_test.reshape(*decoder_target_data_test.shape, 1)


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz

--2024-09-10 10:03:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.51, 3.162.163.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   367MB/s    in 3.6s    

2024-09-10 10:03:44 (346 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
!gunzip cc.fa.300.vec.gz

--2024-09-10 10:04:16--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.165.83.44, 18.165.83.35, 18.165.83.79, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.165.83.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1258183862 (1.2G) [binary/octet-stream]
Saving to: ‘cc.fa.300.vec.gz’

cc.fa.300.vec.gz    100%[===================>]   1.17G  25.2MB/s    in 50s     

2024-09-10 10:05:06 (24.1 MB/s) - ‘cc.fa.300.vec.gz’ saved [1258183862/1258183862]



In [ ]:
# Load pre-trained FastText vectors for English and Farsi
def load_fasttext_vectors(vector_file, vocab):
    ft_model = KeyedVectors.load_word2vec_format(vector_file, binary=False, limit=None)
    embeddings = {}
    for word in vocab:
        if word in ft_model:
            embeddings[word] = ft_model[word]
        else:
            embeddings[word] = np.random.uniform(-0.1, 0.1, ft_model.vector_size)
    return embeddings, ft_model.vector_size

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 867.9 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.5-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.5-py3-none-any.whl (240 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246562 sha256=615813e905937b81a02d13d790c0c3e8f716ad23589944e0c335fb5c90a13e30
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import fasttext
# Load English and Farsi embeddings
embeddings_en, embedding_dim_en = load_fasttext_vectors('cc.en.300.vec', vocab_en)
embeddings_fa, embedding_dim_fa = load_fasttext_vectors('cc.fa.300.vec', vocab_fa)

In [ ]:
# Create embedding matrices
embedding_matrix_en = np.zeros((len(word2idx_en) + 1, embedding_dim_en))
for word, idx in word2idx_en.items():
    embedding_matrix_en[idx] = embeddings_en.get(word, np.random.uniform(-0.1, 0.1, embedding_dim_en))

In [ ]:
embedding_matrix_fa = np.zeros((len(word2idx_fa) + 1, embedding_dim_fa))
for word, idx in word2idx_fa.items():
    embedding_matrix_fa[idx] = embeddings_fa.get(word, np.random.uniform(-0.1, 0.1, embedding_dim_fa))

In [ ]:
# Define LSTM model for English to Farsi translation

# Encoder
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=len(word2idx_en) + 1,
                              output_dim=embedding_dim_en,
                              weights=[embedding_matrix_en],
                              input_length=max_len_en,
                              trainable=False,  # FastText embeddings are not trainable
                              name='encoder_embedding')(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True, name='encoder_lstm')(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
# Decoder
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=len(word2idx_fa) + 1,
                              output_dim=embedding_dim_fa,
                              weights=[embedding_matrix_fa],
                              input_length=max_len_fa,
                              trainable=False,  # FastText embeddings are not trainable
                              name='decoder_embedding')(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(word2idx_fa) + 1, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
# Add callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

In [ ]:
# Train the model with training data
model.fit([sequences_en_train_padded, sequences_fa_train_padded],
          decoder_target_data_train,
          batch_size=64,
          epochs=10,
          validation_split=0.2,
          callbacks=[early_stopping, model_checkpoint])

Epoch 1/10
148/148 [==============================] - ETA: 0s - loss: 1.9911
Epoch 1: val_loss improved from inf to 1.06324, saving model to best_model.keras
148/148 [==============================] - 82s 531ms/step - loss: 1.9911 - val_loss: 1.0632
Epoch 2/10
148/148 [==============================] - ETA: 0s - loss: 1.0222
Epoch 2: val_loss improved from 1.06324 to 0.99006, saving model to best_model.keras
148/148 [==============================] - 78s 530ms/step - loss: 1.0222 - val_loss: 0.9901
Epoch 3/10
148/148 [==============================] - ETA: 0s - loss: 0.9564
Epoch 3: val_loss improved from 0.99006 to 0.94784, saving model to best_model.keras
148/148 [==============================] - 79s 532ms/step - loss: 0.9564 - val_loss: 0.9478
Epoch 4/10
148/148 [==============================] - ETA: 0s - loss: 0.9142
Epoch 4: val_loss improved from 0.94784 to 0.91089, saving model to best_model.keras
148/148 [==============================] - 79s 535ms/step - loss: 0.9142 - val_l

In [ ]:
# Evaluate the model on test data
test_loss = model.evaluate([sequences_en_test_padded, sequences_fa_test_padded], decoder_target_data_test)
print(f"Test Loss: {test_loss}")

92/92 [==============================] - 13s 145ms/step - loss: 0.7981
Test Loss: 0.7981439828872681


In [ ]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(256,), name='decoder_state_input_h')
decoder_state_input_c = Input(shape=(256,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inference, state_h_inference, state_c_inference = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)

decoder_states = [state_h_inference, state_c_inference]
decoder_outputs_inference = decoder_dense(decoder_outputs_inference)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inference] + decoder_states
)

In [ ]:
def translate_sentence(input_sentence, max_len_fa=max_len_fa):
    # Tokenize and pad the input sentence (English)
    input_sequence = [word2idx_en.get(token, 0) for token in tokenizer_en.tokenize(input_sentence)]
    input_sequence = pad_sequences([input_sequence], maxlen=max_len_en, padding='post')

    # Encode the input sentence using the encoder model
    states_value = encoder_model.predict(input_sequence)

    # Initialize the target sequence with the start token ('[CLS]')
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = word2idx_fa.get('[CLS]', 1)  # Use CLS as start token (adjust if needed)

    # Initialize variables to store the generated words
    translated_sentence = []
    stop_condition = False
    while not stop_condition:
        # Predict the next word and its hidden states
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)

        # Get the predicted word index
        predicted_word_index = np.argmax(output_tokens[0, -1, :])
        predicted_word = idx2word_fa.get(predicted_word_index, '')

        # Append the predicted word to the translated sentence
        translated_sentence.append(predicted_word)

        # Stop if we reach the end token ('[SEP]') or exceed max sentence length
        if predicted_word == '[SEP]' or len(translated_sentence) >= max_len_fa:
            stop_condition = True
        else:
            # Update the target sequence with the predicted word for the next time step
            target_sequence = np.zeros((1, 1))
            target_sequence[0, 0] = predicted_word_index

            # Update the decoder states
            states_value = [h, c]

    # Join the translated sentence into a single string and return it
    return ' '.join([word for word in translated_sentence if word != '[SEP]' and word != '[CLS]'])

In [ ]:
# Example usage:
input_sentence = "Swedish English is music and concerts."
translation = translate_sentence(input_sentence)
print("English:", input_sentence)
print("Farsi Translation:", translation)


1/1 [==============================] - 0s 27ms/step
English: Swedish English is music and concerts.
Farsi Translation: به زبان فنلاندی                                                                                                  
